In [2]:
import numpy as np
import pandas as pd

In [3]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.metrics.pairwise import cosine_similarity

Using TensorFlow backend.


In [4]:
df = pd.read_csv('1.csv')
df = df[['name','rate','location','dish_liked','cuisines','approx_cost','neighbourhood']]
df.drop_duplicates('name', inplace=True)
df.dropna(subset=['cuisines'], inplace=True)

In [5]:
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["cuisines"] = df["cuisines"].apply(lambda text: remove_punctuation(text))
df['addr'] = df['neighbourhood'] +' '+ df['location']
df.set_index('name', inplace=True)
indices = pd.Series(df.index)

In [6]:
df

,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Jalsa,4.1,Banashankari,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...",North Indian Mughlai Chinese,800.0,Banashankari,Banashankari Banashankari
Spice Elephant,4.1,Banashankari,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...",Chinese North Indian Thai,800.0,Banashankari,Banashankari Banashankari
San Churro Cafe,3.8,Banashankari,"Churros, Cannelloni, Minestrone Soup, Hot Choc...",Cafe Mexican Italian,800.0,Banashankari,Banashankari Banashankari
Addhuri Udupi Bhojana,3.7,Banashankari,Masala Dosa,South Indian North Indian,300.0,Banashankari,Banashankari Banashankari
Grand Village,3.8,Basavanagudi,"Panipuri, Gol Gappe",North Indian Rajasthani,600.0,Banashankari,Banashankari Basavanagudi
...,...,...,...,...,...,...,...
Chime - Sheraton Grand Bengaluru Whitefield Hotel &...,4.3,"ITPL Main Road, Whitefield","Cocktails, Pizza, Buttermilk",Finger Food,2500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
The Nest - The Den Bengaluru,3.4,"ITPL Main Road, Whitefield",NaN,Finger Food North Indian Continental,1500.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"
Nawabs Empire,3.2,"ITPL Main Road, Whitefield",NaN,North Indian Chinese Arabian Momos,300.0,Whitefield,"Whitefield ITPL Main Road, Whitefield"


In [7]:
l = [i for i in df['addr']]
l2 = [i for i in df['cuisines']]
l3 = [i for i in df['dish_liked']]

In [8]:
dish=[]
for i in l3:
    if pd.isna(i):
        dish.append('')
    else:
        dish.append(''.join(i.split(',')))

In [80]:
# integer encode the documents
vocab_size = 200
encoded_docs = [one_hot(d, vocab_size) for d in l]
encoded_docs2 = [one_hot(d, vocab_size) for d in l2]
encoded_docs3 = [one_hot(d, vocab_size) for d in dish]

max_length = 10
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
padded_docs2 = pad_sequences(encoded_docs2, maxlen=max_length, padding='post')
padded_docs3 = pad_sequences(encoded_docs3, maxlen=max_length, padding='post')


In [74]:
# define the model
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10, 8)             4000      
_________________________________________________________________
flatten_2 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 4,081
Trainable params: 4,081
Non-trainable params: 0
_________________________________________________________________
None


In [81]:
# fit the model
embd = model.predict(padded_docs)
embd2 = model.predict(padded_docs2)
embd3 = model.predict(padded_docs3)

In [82]:
embd,embd2,embd3

(array([[0.51331264],
        [0.51331264],
        [0.51331264],
        ...,
        [0.5028605 ],
        [0.50198805],
        [0.5038426 ]], dtype=float32),
 array([[0.49778104],
        [0.4971337 ],
        [0.49373737],
        ...,
        [0.502337  ],
        [0.49853623],
        [0.4992097 ]], dtype=float32),
 array([[0.49195522],
        [0.5173729 ],
        [0.5084812 ],
        ...,
        [0.5003814 ],
        [0.5003814 ],
        [0.5003814 ]], dtype=float32))

In [83]:
sim=[]
for i in range(indices.shape[0]):
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]])
    #sim.append([embd[i][0],embd2[i][0],embd3[i][0]]) 
    sim.append([embd[i][0],embd2[i][0],embd3[i][0]])

In [84]:
sim[0]

[0.51331264, 0.49778104, 0.49195522]

In [85]:
def rec(name):
    idx = indices[indices == name].index[0]
    corr,reco = [],[]
    for i in range(len(sim)):
        #corr.append( (cosine_similarity([sim[idx]],[sim[i]])[0][0], i))
        cs = cosine_similarity([sim[idx]],[sim[i]])[0][0]
        if cs>=0.99995:
            corr.append((cs,i))
     
    corr = sorted(corr,key = lambda x:x[0], reverse=True)
    reco = [c[1] for c in corr]
    if idx in reco:reco.remove(idx)
    print('Coverage = ',(len(corr)*100/df.shape[0]),'Percent')
    
    return df.iloc[reco[:11]]
    

    
rec("Jalsa")

Coverage =  4.747267759562842 Percent


,rate,location,dish_liked,cuisines,approx_cost,neighbourhood,addr
name,,,,,,,
Frozen Bottle,4.2,Banashankari,"Berryblast, Waffles, Cheesecake, Salted Caramel",Beverages Desserts Ice Cream,400.0,Banashankari,Banashankari Banashankari
Wanderers,4.0,Kalyan Nagar,"Pizza, Peri Peri Fries, Wheat Beer, Craft Beer...",Continental North Indian South Indian Italian,1500.0,Kalyan Nagar,Kalyan Nagar Kalyan Nagar
Thamboola,3.9,Banashankari,"Spring Roll, Manchow Soup, Babycorn Manchurian...",North Indian Chinese Continental,800.0,Banashankari,Banashankari Banashankari
Hotel Mangala,3.9,Banashankari,"Khali Dosa, Kali Dosa, Masala Dosa, Vada Sambar",South Indian,250.0,Banashankari,Banashankari Banashankari
Laziz Pizza,3.8,Koramangala 7th Block,"Pizza, Peri Peri Fries, Veg Paprika",Fast Food Burger Pizza,400.0,BTM,BTM Koramangala 7th Block
BIB-Breakfast in the Box,3.7,Whitefield,"French Fries, Burgers, English Breakfast, Panc...",American Sandwich,600.0,Brookefield,Brookefield Whitefield
Delhi Mess,4.1,New BEL Road,"Aloo Paratha, Roti, Butter Chicken",North Indian Chinese,300.0,New BEL Road,New BEL Road New BEL Road
Domino's Pizza,3.9,Banashankari,"Pane Pizza, Lava Cake, Pasta, Stuffed Garlic B...",Pizza Fast Food,800.0,Banashankari,Banashankari Banashankari
Krazzy Folds,4.2,Jayanagar,"Pasta, Tacos, Macaroni, Panna Cotta, Salads, S...",Cafe Continental Sandwich Beverages,800.0,Basavanagudi,Basavanagudi Jayanagar


In [86]:
cosine_similarity([sim[11]],[sim[0]])[0][0]

0.999984